# Testing Agent Components

I used this notebook to test different tools and components.  

In [ ]:
from agents import manager_agent
manager_agent.visualize()

In [ ]:
from PIL import Image

#manager_agent.run("How many bird species are in this image?", images=images)
manager_agent.run("In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?")

In [ ]:
manager_agent.run('How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.')

In [ ]:
DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
question  = "What is the final numeric output from the attached Python code?"
task_file_path = "f918266a-b3e0-4914-865d-4faa564f1aef"
filename = "f918266a-b3e0-4914-865d-4faa564f1aef.py"
context = f"""
Here is the question you need to answer: {question}

This question has an associated file. You can download the file from 
{DEFAULT_API_URL}/files/{task_file_path}
using the download_file_from_url tool and name it {filename}.  
After downloading the file, use the save_and_read_file tool to read the contents of the file.

Come up with a plan to analyze the file content above to answer the question.  Execute that plan carefully and stay focused.
"""
full_prompt = f"""{context}

When answering, provide ONLY the precise answer requested. 
Do not include explanations, steps, reasoning, or additional text.
Be direct and specific. GAIA benchmark requires exact matching answers.
For example, if asked "What is the capital of France?", respond simply with "Paris".
"""
manager_agent.run(context)

In [ ]:
DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
question  = "Hi, I was out sick from my classes on Friday, so I'm trying to figure out what I need to study for my Calculus mid-term next week. My friend from class sent me an audio recording of Professor Willowbrook giving out the recommended reading for the test, but my headphones are broken :(Could you please listen to the recording for me and tell me the page numbers I'm supposed to go over? I've attached a file called Homework.mp3 that has the recording. Please provide just the page numbers as a comma-delimited list. And please provide the list in ascending order."
task_file_path = "1f975693-876d-457b-a649-393859e79bf3"
filename = "1f975693-876d-457b-a649-393859e79bf3.mp3"
context = f"""
Here is the question you need to answer: {question}

This question has an associated file. You can download the file from 
{DEFAULT_API_URL}/files/{task_file_path}
using the download_file_from_url tool and name it {filename}.  
After downloading the file, use the save_and_read_file tool to read the contents of the file.

Come up with a plan to analyze the file content above to answer the question.  Execute that plan carefully and stay focused.
"""
full_prompt = f"""{context}

When answering, provide ONLY the precise answer requested. 
Do not include explanations, steps, reasoning, or additional text.
Be direct and specific. GAIA benchmark requires exact matching answers.
For example, if asked "What is the capital of France?", respond simply with "Paris".
"""
manager_agent.run(context)

In [ ]:
DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
question  = "The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places."
task_file_path = "7bd855d8-463d-4ed5-93ca-5fe35145f733"
filename = "7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx"
context = f"""
Here is the question you need to answer: {question}

This question has an associated file. You can download the file from 
{DEFAULT_API_URL}/files/{task_file_path}
using the download_file_from_url tool and name it {filename}.  
After downloading the file, use the save_and_read_file tool to read the contents of the file.

Come up with a plan to analyze the file content above to answer the question.  Execute that plan carefully and stay focused.
"""
full_prompt = f"""{context}

When answering, provide ONLY the precise answer requested. 
Do not include explanations, steps, reasoning, or additional text.
Be direct and specific. GAIA benchmark requires exact matching answers.
For example, if asked "What is the capital of France?", respond simply with "Paris".
"""
manager_agent.run(context)

In [ ]:
manager_agent.run("How many r's are in the word Strawberry?")


In [ ]:
from tools.tools import youtube_frames_to_images
results = youtube_frames_to_images('https://youtube.com/watch?v=L1vXCYZAYYM')

In [ ]:
from tools.tools import youtube_transcribe
results = youtube_transcribe('https://www.youtube.com/watch?v=1htKBjuUWec')
print(results)

In [ ]:
from PIL import Image
from os import walk, path

# Path to image file
picture_path = './images'

# List to hold images
images = []

# Walk through the directory and get all .png files paths
for root, dirs, files in walk(picture_path):
    for file in files:
        if file.endswith('.png'):
            # Construct full path of each png file
            picture_file = path.join(root, file)
            
            try:
                # Open image from path
                img = Image.open(picture_file)
                
                # Append it to the list of images after converting it to RGB format
                images.append(img.convert('RGB'))
            except IOError as e:
                print(f'Error opening {picture_file}: {e}')
                continue


from tools.tools import use_vision_model

query = "Review these pictures.  What is the highest number of bird species to appear in the same photo of these photos?"

tool_input = {
    "question": query,
    "images": images
}

use_vision_model(**tool_input)

In [ ]:
from tools.tools import download_file_from_url, transcribe_audio

task_file_path = "1f975693-876d-457b-a649-393859e79bf3"
filename = "1f975693-876d-457b-a649-393859e79bf3.mp3"
url = f'{DEFAULT_API_URL}/files/{task_file_path}'

tool_input = {"url": url}

filename = download_file_from_url(**tool_input).strip("File downloaded to ").strip(". You can now process this file.'")

tool_input = {"audio_file_path": filename}

transcribe_audio(**tool_input)

In [ ]:
from tools.tools import read_file, download_file_from_url

DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
taskid = "f918266a-b3e0-4914-865d-4faa564f1aef"
filename = f"{taskid}.py"
url = f'{DEFAULT_API_URL}/files/{taskid}'

tool_input = {"url": url, "filename": filename}

filepath = download_file_from_url(**tool_input).strip("File downloaded to ").strip(". You can now process this file.'")


tool_input = {"filepath": filepath}

output = read_file(**tool_input)

print(output)